In [24]:
import utils
import params
import numpy as np

from tqdm import tqdm

In [2]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    #n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = [float(x) for x in tokens[1:]]
    return data

In [3]:
fname = 'MODEL/wiki-news-300d-1M.vec'
vectors = load_vectors(fname)

In [92]:
def AGG_CAPTIONS(captions):
    from collections import defaultdict
    recaptions = defaultdict(list)
    for caption in captions:
        name = caption.get('image_id')
        des = caption.get('caption').lower()       
        recaptions[name].append(des)
    return recaptions

In [19]:
def reCaption(captions):
    from collections import defaultdict
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    recaptions = defaultdict(list)
    stop_words = set (stopwords.words( 'english' ))
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()

    for caption in captions:
        name = caption.get('image_id')
        des = caption.get('caption').lower()

        #remove stopwords
        des_stemmed = []
        for word in analyze(des):
            if word not in stop_words:
                des_stemmed.append(word)
        des_stemmed = ' '.join(des_stemmed)
        
        recaptions[name].append(des_stemmed)
    return recaptions

In [112]:
captions = utils.readCaptions(params.VAL_CAPTIONS_PATH)

In [113]:
#agg = AGG_CAPTIONS(captions)

In [114]:
recaptions = reCaption(captions)

In [115]:
def applyword2vec(captions, vectors, path):
    from sklearn.feature_extraction.text import CountVectorizer
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()
    
    i = 0
    for caption in tqdm(captions.keys()):

        sentences = captions[caption]
        caption_descriptors = []
        for sentence in sentences:
            word_descriptor = []
            for word in analyze(sentence):
                if vectors.get(word) is not None:
                    word_descriptor.append( np.asarray(vectors[word] ))
                else:
                    word_descriptor.append( np.zeros(300))

            sentence_descriptor = np.mean(word_descriptor, axis=0)
            caption_descriptors.append(sentence_descriptor)
            name = path+str(caption)+'.npy'
        np.save(name,np.asarray(caption_descriptors))    

In [116]:
applyword2vec(recaptions, vectors, params.W2V_DESCRIPTORS_VAL_PATH)

100%|██████████| 40504/40504 [03:24<00:00, 197.62it/s]
